# 1. 환경세팅
requirements: conda

밑의 모든 코드는 EmoSDS/ 디렉토리에서 실행할 것

## 1-1. Llama 3.2 3B 모델 다운로드

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

!mkdir llama/3_2/3B
!cd llama/3_2/3B
!git clone https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
!cd ../../../

## 1-2. kmeans 모델 다운로드

In [ ]:
# >> 여기서 pt파일 다운받아서 utils/speech2unit 에 저장할 것.
# https://www.notion.so/jhwanlee/Future-Internet-16dd543c7357803d96e6f9b7914048a3?pvs=4#175d543c735780dfac64fdd9e5ebe44a

## 1-3. conda 환경 생성

In [ ]:
!conda create --name emosds python=3.8
!conda activate emosds
!conda install pip=24.0
!conda install cudatoolkit=11.7 -c conda-forge
!pip install -r requirements.txt
!pip install -U datasets
# pip install transformers==4.34
!pip install omegaconf==2.3.0
!pip install accelerate==0.26.0
!pip install torch==2.4.1
!pip install torchaudio==2.4.1
!pip install transformers==4.45
!pip install --upgrade huggingface-hub
!pip install bert_score
!pip install seaborn
!pip install nltk rouge-score

# 2. 데이터 Preprocess(현재는 skip해도 됨)

## 2-0. 데이터셋 다운로드

In [ ]:
# >> StyleTalk 다운로드 --> data/styletalk/에 압축 해제할 것
#    밑에 train.csv, eval.csv 파일도 다운받아서 data/styletalk에 저장할 것
# https://www.notion.so/jhwanlee/Future-Internet-16dd543c7357803d96e6f9b7914048a3?pvs=4#17dd543c735780eeb32cf35fd4689f0f

## 2-1. ASR task 데이터 생성

In [ ]:
# ! 실행하기 전에 output path 잘 확인할 것
!python3 utils/build_data.py asr --librispeech-dir /shared/NAS_SSD/wooyeol/LibriSpeech/train-clean-100/train-clean-100/
# python3 utils/merge_jsonl_files.py data/asr/layer6_k1000/asr_task_styletalk_train.jsonl data/asr/layer6_k1000/asr_task_styletalk_eval.jsonl data/asr/layer6_k1000/asr_task_styletalk_train_eval.jsonl
# python3 utils/merge_jsonl_files.py data/asr/layer6_k1000/asr_task_styletalk_train_eval.jsonl data/asr/layer6_k1000/asr_task_librispeech.jsonl data/asr/layer6_k1000/asr_task_librispeech_styletalk.jsonl

!python3 utils/temp_diversify_prompt.py /home/jhwan98/EmoSDS/data/asr/layer6_k2048_merged/asr_task_librispeech.jsonl /home/jhwan98/EmoSDS/data/asr/layer6_k2048_merged/asr_task_librispeech_diverse_prompt.jsonl --prompts-path /home/jhwan98/EmoSDS/data/asr/prompts_for_asr.txt

## 2-2. ASR+SER task 데이터 생성

In [ ]:
# ASR+SER
# ! 실행하기 전에 output path 잘 확인할 것
!python3 utils/build_data.py asr+ser --dailytalk-dir /shared/NAS_SSD/jhl/futureinternet/dailytalk --esd-dir /shared/NAS_SSD/jhl/futureinternet/EmotionSpeechDataset --ravdess-dir /shared/NAS_SSD/jhl/futureinternet/ravdess_audio --crema-dir /shared/NAS_SSD/jhl/futureinternet/CREMA-D/AudioWAV
!CUDA_VISIBLE_DEVICES=1 python3 utils/build_data.py asr+ser --dailytalk-dir /shared/NAS_SSD/jhl/futureinternet/dailytalk
!CUDA_VISIBLE_DEVICES=1 python3 utils/build_data.py asr+ser --styletalk-dir /shared/NAS_SSD/jhl/futureinternet/styletalk
!python3 utils/build_data.py asr+ser --esd-dir /shared/NAS_SSD/jhl/futureinternet/EmotionSpeechDataset
# python3 utils/build_data.py asr+ser --crema-dir /shared/NAS_SSD/jhl/futureinternet/CREMA-D/AudioWAV
!python3 utils/merge_jsonl_files.py data/asr_ser/layer6_k2048/merged/asr_ser_task_dailytalk.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_esd.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_dailytalk_esd.jsonl
!python3 utils/merge_jsonl_files.py data/asr_ser/layer6_k2048/merged/asr_ser_task_dailytalk_esd.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_ravdess.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_merged.jsonl
!python3 utils/merge_jsonl_files.py data/asr_ser/layer6_k2048/merged/asr_ser_task_merged.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_crema.jsonl data/asr_ser/layer6_k2048/merged/asr_ser_task_merged2.jsonl
!python3 utils/asr_ser_split_jsonl_dataset.py data/asr_ser/layer7_k2000/merged/asr_ser_task_merged_no_dailytalk.jsonl data/asr_ser/layer7_k2000/merged
# python3 utils/asr_ser_split_jsonl_dataset.py data/asr_ser/layer7_k2000/merged/asr_ser_task_esd.jsonl data/asr_ser/layer7_k2000/merged
# python3 utils/asr_ser_split_jsonl_dataset.py data/asr_ser/layer7_k2000/merged/splitted_task/asr_ser_task_merged.jsonl data/asr_ser/layer7_k2000/merged/splitted_task
# python3 utils/asr_ser_split_jsonl_dataset.py data/asr_ser/layer6_k1000/merged/asr_ser_task_esd_residual.jsonl data/asr_ser/layer6_k1000/merged

!python3 utils/merge_jsonl_files.py data/asr_ser/layer7_k2000/merged/asr_ser_task_esd.jsonl data/asr_ser/layer7_k2000/merged/asr_ser_task_ravdess.jsonl data/asr_ser/layer7_k2000/merged/asr_ser_task_esd_ravdess.jsonl
!python3 utils/merge_jsonl_files.py data/asr_ser/layer7_k2000/merged/asr_ser_task_esd_ravdess.jsonl data/asr_ser/layer7_k2000/merged/asr_ser_task_crema.jsonl data/asr_ser/layer7_k2000/merged/asr_ser_task_merged_no_dailytalk.jsonl

!python3 utils/merge_jsonl_files.py data/asr_ser/layer7_k2000/merged/splitted_task/asr_ser_task_esd.jsonl data/asr_ser/layer7_k2000/merged/splitted_task/asr_ser_task_dailytalk.jsonl data/asr_ser/layer7_k2000/merged/splitted_task/asr_ser_task_merged.jsonl

!python3 utils/temp_diversify_prompt.py /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_train_balanced_dailytalk_newresidual.jsonl /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_train_balanced_dailytalk_newresidual_diverse_prompt.jsonl --prompts-path /home/jhwan98/EmoSDS/data/asr_ser/prompts_for_asr_ser.txt --asr-ser
!python3 utils/temp_diversify_prompt.py /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_test_balanced_dailytalk_newresidual.jsonl /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_test_balanced_dailytalk_newresidual_diverse_prompt.jsonl --prompts-path /home/jhwan98/EmoSDS/data/asr_ser/prompts_for_asr_ser.txt --asr-ser
!python3 utils/temp_diversify_prompt.py /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_valid_balanced_dailytalk_newresidual.jsonl /home/jhwan98/EmoSDS/data/asr_ser/layer6_k1000/merged/asr_ser_task_valid_balanced_dailytalk_newresidual_diverse_prompt.jsonl --prompts-path /home/jhwan98/EmoSDS/data/asr_ser/prompts_for_asr_ser.txt --asr-ser

## 2-2. Unified(ASR + Style Prediction + Response Prediction) task 데이터 생성

In [ ]:
# ! 실행하기 전에 output path, speech2unit 잘 확인할 것
!python3 utils/build_data.py unified --dailytalk-dir /shared/NAS_SSD/jhl/futureinternet/dailytalk --ravdess-dir /shared/NAS_SSD/jhl/futureinternet/ravdess_audio --ravdess-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/ravdess_without_audio.jsonl --crema-dir /shared/NAS_SSD/jhl/futureinternet/CREMA-D/AudioWAV --crema-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/crema_without_audio.jsonl --esd-dir /shared/NAS_SSD/jhl/futureinternet/EmotionSpeechDataset --esd-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/esd_without_audio.jsonl
# python3 utils/build_data.py unified --ravdess-dir /shared/NAS_SSD/jhl/futureinternet/ravdess_audio --ravdess-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/ravdess_without_audio.jsonl --crema-dir /shared/NAS_SSD/jhl/futureinternet/CREMA-D/AudioWAV --crema-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/crema_without_audio.jsonl
# python3 utils/build_data.py unified --crema-dir /shared/NAS_SSD/jhl/futureinternet/CREMA-D/AudioWAV --crema-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/crema_without_audio.jsonl
# python3 utils/build_data.py unified --ravdess-dir /shared/NAS_SSD/jhl/futureinternet/ravdess_audio --ravdess-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/ravdess_without_audio.jsonl
# python3 utils/build_data.py unified --esd-dir /shared/NAS_SSD/jhl/futureinternet/EmotionSpeechDataset --esd-syn-path /home/jhwan98/EmoSDS/data/synthesized/unified/esd_without_audio_5emotions.jsonl
# python3 utils/build_data.py unified --styletalk-dir /shared/NAS_SSD/jhl/futureinternet/styletalk

!python3 utils/merge_jsonl_files.py data/unified/layer7_k2000_merged2/unified_task_dailytalk_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_ravdess_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_dailytalk_ravdess_train_balanced.jsonl
!python3 utils/merge_jsonl_files.py data/unified/layer7_k2000_merged2/unified_task_dailytalk_ravdess_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_crema_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_merged_train_balanced.jsonl
!python3 utils/merge_jsonl_files.py data/unified/layer7_k2000_merged2/unified_task_merged_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_esd_train_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_merged2_train_balanced.jsonl

!python3 utils/merge_jsonl_files.py data/unified/layer7_k2000_merged2/unified_task_dailytalk_test_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_esd_test_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_merged2_test_balanced.jsonl

!python3 utils/merge_jsonl_files.py data/unified/layer7_k2000_merged2/unified_task_dailytalk_valid_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_esd_valid_balanced.jsonl data/unified/layer7_k2000_merged2/unified_task_merged2_valid_balanced.jsonl

# >> ASR 부분 제외
!python3 utils/temp_remove_asr_part.py /home/jhwan98/EmoSDS/data/unified/layer7_k2000_merged2/unified_task_esd2_5emotions_train_balanced.jsonl /home/jhwan98/EmoSDS/data/unified/layer7_k2000_merged2/unified_task_esd2_5emotions_train_balanced_asr_removed.jsonl

# 3. ASR task 훈련 진행

In [ ]:
# ! overwrite_output_dir 여부 확인
!bash scripts/asr_sft.sh

In [ ]:
# >> 모델 zip파일로 저장(파일만 저장함)
# ! 실행하기 전에 zip 파일명 바꿀 것

!find /shared/NAS_SSD/jhl/futureinternet/output/asr_6layer_k1000_merged -type f -maxdepth 1 -exec zip asr_training.zip {} +
!find /shared/NAS_SSD/jhl/futureinternet/output/asr -type f -maxdepth 1 -exec zip asr_training_10epochs_20250124.zip {} +
!find ./checkpoint-1115 -maxdepth 1 -type f -exec zip asr_training_k2048_10epochs_20250201.zip {} +

# 4. ASR+SER task 훈련 진행

In [ ]:
# ! overwrite_output_dir 여부 확인
!bash scripts/asr_ser_sft.sh
# bash scripts/asr_ser_sft_splitted.sh

In [ ]:
# >> 모델 zip파일로 저장(파일만 저장함)
# ! 실행하기 전에 zip 파일명 바꿀 것

!find /shared/NAS_SSD/jhl/futureinternet/output/asr_ser_6layer_k1000_diverse_prompt_only_esd_residual -type f -maxdepth 1 -exec zip asr_ser_training.zip {} +
!find /shared/NAS_SSD/jhl/futureinternet/output/asr_ser -type f -maxdepth 1 -exec zip asr_ser_training_3epochs_20250121.zip {} +
!find ./asr_ser_7layer_k2000_merged/checkpoint-267 -type f -maxdepth 1 -exec zip asr_ser_training_k2000_10epochs_20250128.zip {} +

# 5. Unified(ASR + Style Prediction + Response Prediction) task 훈련 진행

In [ ]:
# ! overwrite_output_dir 여부 확인
!bash scripts/unified_sft.sh

In [ ]:
# >> 모델 zip파일로 저장(파일만 저장함)
# ! 실행하기 전에 zip 파일명 바꿀 것

!find /shared/NAS_SSD/jhl/futureinternet/output/unified -type f -maxdepth 1 -exec zip unified_training_6epochs_20250113.zip {} +

# 5. CLI inference 수행(Optional)

In [ ]:
# ! speech2unit 확인할 것(no-interact일 경우 상관없음)
# ! cli_infer_emosds의 output path, residual length 확인할 것
!CUDA_VISIBLE_DEVICES=0 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/for_confusion" --input-path "/home/jhwan98/EmoSDS/data/for_confusion_matrix.txt" --output-dir "output" --no-interact --confusion
!CUDA_VISIBLE_DEVICES=0 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/unified_7layer_k2000_merged/checkpoint-192" --input-path "/home/jhwan98/EmoSDS/data/unified/layer7_k2000_merged/unified_task_merged_test_balanced_asr_removed.jsonl" --output-dir "output" --no-interact --confusion

!CUDA_VISIBLE_DEVICES=0,2 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/unified_6layer_k1000_merged_only_esd_residual_only_type1_20250207" --input-path "/home/jhwan98/EmoSDS/data/unified/layer6_k1000_merged/unified_task_esd_5emotions_test_balanced_type1.jsonl" --output-dir "output" --no-interact --no-confusion
!CUDA_VISIBLE_DEVICES=0,2 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/unified_6layer_k1000_merged_esd_balanced_residual_type3_20250216" --input-path "/home/jhwan98/EmoSDS/data/unified/layer6_k1000_merged/unified_task_esd_test_balanced_type3_20250216.jsonl" --output-dir "output" --no-interact --confusion
!CUDA_VISIBLE_DEVICES=0,2 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/unified_6layer_k1000_merged_dailytalk_balanced_5emotions_newresidual_20250215" --input-path "/home/jhwan98/EmoSDS/data/unified/layer6_k1000_merged/unified_task_dailytalk_test_balanced_newresidual_2000_no883.jsonl" --output-dir "output" --no-interact --confusion
!CUDA_VISIBLE_DEVICES=1 python3 src/infer/cli_infer_emosds.py --model-path "/shared/NAS_SSD/jhl/futureinternet/output/asr_ser_6layer_k1000_diverse_prompt_only_esd_residual" --output-dir "output" --interact --no-confusion

# 입력 prompt 형식: "[History] <<Input>>: [wav파일path(assets/testprompts/에 있는 거)]"